# Data Cleaning

` Data cleaning is a time consuming and unenjoyable task, yet it's a very important one. Keep in mind, "garbage in, garbage out".`

#### Feeding dirty data into a model will give us results that are meaningless.

### Objective:

1. Getting the data 
2. Cleaning the data 
3. Organizing the data - organize the cleaned data into a way that is easy to input into other algorithms

### Output :
#### cleaned and organized data in two standard text formats:

1. Corpus - a collection of text
2. Document-Term Matrix - word counts in matrix format

## Problem Statement

Look at transcripts of various comedians and note their similarities and differences and find if the stand up comedian of your choice has comedy style different than other comedian.


## Getting The Data

You can get the transcripts of some comedian from [Scraps From The Loft](http://scrapsfromtheloft.com). 

You can take help of IMDB and select only 10 or 20 comedian having highest rating.






### For example:

In [1]:
# Web scraping, pickle imports
import requests
from bs4 import BeautifulSoup
import pickle

# Scrapes transcript data from scrapsfromtheloft.com
def url_to_transcript(url):
    '''Returns transcript data specifically from scrapsfromtheloft.com.'''
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    # text = [p.text for p in soup.find(class_="post-content")]
    text = [p.text for p in soup.find('p', attrs={'style': 'text-align: justify;'})]
    print(url)
    return text

# URLs of transcripts in scope
urls = ['http://scrapsfromtheloft.com/2017/05/06/louis-ck-oh-my-god-full-transcript/',
        'http://scrapsfromtheloft.com/2017/04/11/dave-chappelle-age-spin-2017-full-transcript/',
        'http://scrapsfromtheloft.com/2018/03/15/ricky-gervais-humanity-transcript/',
        # 'http://scrapsfromtheloft.com/2017/08/07/bo-burnham-2013-full-transcript/',
        'http://scrapsfromtheloft.com/2017/05/24/bill-burr-im-sorry-feel-way-2014-full-transcript/',
        'http://scrapsfromtheloft.com/2017/04/21/jim-jefferies-bare-2014-full-transcript/',
        'http://scrapsfromtheloft.com/2017/08/02/john-mulaney-comeback-kid-2015-full-transcript/',
        # 'http://scrapsfromtheloft.com/2017/10/21/hasan-minhaj-homecoming-king-2017-full-transcript/',
        'http://scrapsfromtheloft.com/2017/09/19/ali-wong-baby-cobra-2016-full-transcript/',
        'http://scrapsfromtheloft.com/2017/08/03/anthony-jeselnik-thoughts-prayers-2015-full-transcript/',
        'http://scrapsfromtheloft.com/2018/03/03/mike-birbiglia-my-girlfriends-boyfriend-2013-full-transcript/',
        'http://scrapsfromtheloft.com/2017/08/19/joe-rogan-triggered-2016-full-transcript/']

# Comedian names
comedians = ['louis', 'dave', 'ricky', 'bill', 'jim', 'john', 'ali', 'anthony', 'mike', 'joe']

Two links removed as they were no longer available

In [2]:
# # Actually request transcripts (takes a few minutes to run)
transcripts = [url_to_transcript(u) for u in urls]

http://scrapsfromtheloft.com/2017/05/06/louis-ck-oh-my-god-full-transcript/
http://scrapsfromtheloft.com/2017/04/11/dave-chappelle-age-spin-2017-full-transcript/
http://scrapsfromtheloft.com/2018/03/15/ricky-gervais-humanity-transcript/
http://scrapsfromtheloft.com/2017/05/24/bill-burr-im-sorry-feel-way-2014-full-transcript/
http://scrapsfromtheloft.com/2017/04/21/jim-jefferies-bare-2014-full-transcript/
http://scrapsfromtheloft.com/2017/08/02/john-mulaney-comeback-kid-2015-full-transcript/
http://scrapsfromtheloft.com/2017/09/19/ali-wong-baby-cobra-2016-full-transcript/
http://scrapsfromtheloft.com/2017/08/03/anthony-jeselnik-thoughts-prayers-2015-full-transcript/
http://scrapsfromtheloft.com/2018/03/03/mike-birbiglia-my-girlfriends-boyfriend-2013-full-transcript/
http://scrapsfromtheloft.com/2017/08/19/joe-rogan-triggered-2016-full-transcript/


In [24]:
# # Pickle files for later use

# # Make a new directory to hold the text files
# !mkdir transcripts_self

# for i, c in enumerate(comedians):
#     with open("transcripts_self/" + c + ".txt", "wb") as file:
#         pickle.dump(transcripts[i], file)

In [3]:
# Load pickled files
data = {}
for i, c in enumerate(comedians):
    with open("transcripts_self/" + c + ".txt", "rb") as file:
        data[c] = pickle.load(file)

In [4]:
# Double check to make sure data has been loaded properly
data.keys()

dict_keys(['louis', 'dave', 'ricky', 'bill', 'jim', 'john', 'ali', 'anthony', 'mike', 'joe'])

In [5]:
# More checks
data['louis'][:2]

['Intro', '']

## Cleaning The Data

When dealing with numerical data, data cleaning often involves removing null values and duplicate data, dealing with outliers, etc. With text data, there are some common data cleaning techniques, which are also known as text pre-processing techniques.

With text data, this cleaning process can go on forever. There's always an exception to every cleaning step. So, we're going to follow the MVP (minimum viable product) approach - start simple and iterate.
### Assignment:
1. Perform the following data cleaning on transcripts:
i) Make text all lower case
ii) Remove punctuation
iii) Remove numerical values
iv) Remove common non-sensical text (/n)
v) Tokenize text
vi) Remove stop words

In [8]:
# Let's take a look at our data again
next(iter(data.keys()))

'louis'

In [9]:
# Notice that our dictionary is currently in key: comedian, value: list of text format
next(iter(data.values()))

['Intro',
 '',
 '\nFade the music out. Let’s roll. Hold there. Lights. Do the lights. Thank you. Thank you very much. I appreciate that. I don’t necessarily agree with you, but I appreciate very much. Well, this is a nice place. This is easily the nicest place For many miles in every direction. That’s how you compliment a building And shit on a town with one sentence. It is odd around here, as I was driving here. There doesn’t seem to be any difference Between the sidewalk and the street for pedestrians here. People just kind of walk in the middle of the road. I love traveling And seeing all the different parts of the country. I live in New York. I live in a– There’s no value to your doing that at all.']

In [6]:
# We are going to change this to key: comedian, value: string format
def combine_text(list_of_text):
    '''Takes a list of text and combines them into one large chunk of text.'''
    combined_text = ' '.join(list_of_text)
    return combined_text

In [7]:
# Combine it!
data_combined = {key: [combine_text(value)] for (key, value) in data.items()}

In [8]:
# We can either keep it in dictionary format or put it into a pandas dataframe
import pandas as pd
pd.set_option('max_colwidth',150)

data_df = pd.DataFrame.from_dict(data_combined).transpose()
data_df.columns = ['transcript']
data_df = data_df.sort_index()
data_df

,transcript
ali,"Ladies and gentlemen, please welcome to the stage: Ali Wong! Hi. Hello! Welcome! Thank you! Thank you for coming. Hello! Hello. We are gonna have ..."
anthony,"Thank you. Thank you. Thank you, San Francisco . Thank you so much. So good to be here. People were surprised when I told ’em I was gonna tape my..."
bill,"[cheers and applause] All right, thank you! Thank you very much! Thank you. Thank you. Thank you. How are you? What’s going on? Thank you. It’s a ..."
dave,This is Dave. He tells dirty jokes for a living. That stare is where most of his hard work happens. It signifies a profound train of thought...
jim,"[Car horn honks] [Audience cheering] [Announcer] Ladies and gentlemen, please welcome to the stage Mr. Jim Jefferies! [Upbeat music playing] Hello..."
joe,[rock music playing]
john,"All right, Petunia. Wish me luck out there. You will die on August 7th, 2037. That’s pretty good. All right. Hello. Hello, Chicago. Nice to see yo..."
louis,Intro \nFade the music out. Let’s roll. Hold there. Lights. Do the lights. Thank you. Thank you very much. I appreciate that. I don’t necessarily...
mike,"Wow. Hey, thank you. Thanks. Thank you, guys. Hey, Seattle. Nice to see you. Look at this. Look at us. We’re here. This is crazy. It’s insane. So ..."
ricky,Hello. Hello! How you doing? Great. Thank you. Wow. Calm down. Shut the fuck up. Thank you.


In [9]:
# Let's take a look at the transcript for Ali Wong
data_df.transcript.loc['ali']

'Ladies and gentlemen, please welcome to the stage: Ali Wong! Hi. Hello! Welcome! Thank you! Thank you for coming. Hello! Hello. We are gonna have to get this shit over with, ’cause I have to pee in, like, ten minutes. But thank you, everybody, so much for coming.'

In [10]:
# Apply a first round of text cleaning techniques
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    # text = re.sub('\[.*?\]', '', text)
    # text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    # text = re.sub('\w*\d\w*', '', text)
    text = re.sub(r'[^a-zA-Z]', ' ', text) 
    return text

round1 = lambda x: clean_text_round1(x)

Removed everything except letters

In [11]:
# Let's take a look at the updated text
data_clean = pd.DataFrame(data_df.transcript.apply(round1))
data_clean.transcript.loc['ali']

'ladies and gentlemen  please welcome to the stage  ali wong  hi  hello  welcome  thank you  thank you for coming  hello  hello  we are gonna have to get this shit over with   cause i have to pee in  like  ten minutes  but thank you  everybody  so much for coming '

In [12]:
# Apply a second round of cleaning
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    test = re.sub(r'[^a-zA-Z]', '', text) # added new regex
    return text

round2 = lambda x: clean_text_round2(x)

In [13]:
# Let's take a look at the updated text
data_clean = pd.DataFrame(data_clean.transcript.apply(round2))
data_clean

,transcript
ali,ladies and gentlemen please welcome to the stage ali wong hi hello welcome thank you thank you for coming hello hello we are gonna have ...
anthony,thank you thank you thank you san francisco thank you so much so good to be here people were surprised when i told em i was gonna tape my...
bill,cheers and applause all right thank you thank you very much thank you thank you thank you how are you what s going on thank you it s a ...
dave,this is dave he tells dirty jokes for a living that stare is where most of his hard work happens it signifies a profound train of thought...
jim,car horn honks audience cheering announcer ladies and gentlemen please welcome to the stage mr jim jefferies upbeat music playing hello...
joe,rock music playing
john,all right petunia wish me luck out there you will die on august th that s pretty good all right hello hello chicago nice to see yo...
louis,intro fade the music out let s roll hold there lights do the lights thank you thank you very much i appreciate that i don t necessarily ...
mike,wow hey thank you thanks thank you guys hey seattle nice to see you look at this look at us we re here this is crazy it s insane so ...
ricky,hello hello how you doing great thank you wow calm down shut the fuck up thank you


In [14]:
data_clean.transcript.loc['ali']

'ladies and gentlemen  please welcome to the stage  ali wong  hi  hello  welcome  thank you  thank you for coming  hello  hello  we are gonna have to get this shit over with   cause i have to pee in  like  ten minutes  but thank you  everybody  so much for coming '

In [15]:
import nltk

Tokenizing sentences to words

In [16]:
data_token = pd.DataFrame(data_clean.transcript.apply(lambda x: nltk.word_tokenize(x)))
data_token

,transcript
ali,"[ladies, and, gentlemen, please, welcome, to, the, stage, ali, wong, hi, hello, welcome, thank, you, thank, you, for, coming, hello, hello, we, ar..."
anthony,"[thank, you, thank, you, thank, you, san, francisco, thank, you, so, much, so, good, to, be, here, people, were, surprised, when, i, told, em, i, ..."
bill,"[cheers, and, applause, all, right, thank, you, thank, you, very, much, thank, you, thank, you, thank, you, how, are, you, what, s, going, on, tha..."
dave,"[this, is, dave, he, tells, dirty, jokes, for, a, living, that, stare, is, where, most, of, his, hard, work, happens, it, signifies, a, profound, ..."
jim,"[car, horn, honks, audience, cheering, announcer, ladies, and, gentlemen, please, welcome, to, the, stage, mr, jim, jefferies, upbeat, music, play..."
joe,"[rock, music, playing]"
john,"[all, right, petunia, wish, me, luck, out, there, you, will, die, on, august, th, that, s, pretty, good, all, right, hello, hello, chicago, nice, ..."
louis,"[intro, fade, the, music, out, let, s, roll, hold, there, lights, do, the, lights, thank, you, thank, you, very, much, i, appreciate, that, i, don..."
mike,"[wow, hey, thank, you, thanks, thank, you, guys, hey, seattle, nice, to, see, you, look, at, this, look, at, us, we, re, here, this, is, crazy, it..."
ricky,"[hello, hello, how, you, doing, great, thank, you, wow, calm, down, shut, the, fuck, up, thank, you]"


Removing Stopwords

In [17]:
data_sw = pd.DataFrame(data_token.transcript.apply(lambda x: [word for word in x if not word in nltk.corpus.stopwords.words('english')]))
data_sw

,transcript
ali,"[ladies, gentlemen, please, welcome, stage, ali, wong, hi, hello, welcome, thank, thank, coming, hello, hello, gon, na, get, shit, cause, pee, lik..."
anthony,"[thank, thank, thank, san, francisco, thank, much, good, people, surprised, told, em, gon, na, tape, special, san, francisco, said, would, politic..."
bill,"[cheers, applause, right, thank, thank, much, thank, thank, thank, going, thank, pleasure, greater, atlanta, georgia, area, oasis, nice, know, cam..."
dave,"[dave, tells, dirty, jokes, living, stare, hard, work, happens, signifies, profound, train, thought, alchemist, fire, transforms, fear, tragedy, l..."
jim,"[car, horn, honks, audience, cheering, announcer, ladies, gentlemen, please, welcome, stage, mr, jim, jefferies, upbeat, music, playing, hello, si..."
joe,"[rock, music, playing]"
john,"[right, petunia, wish, luck, die, august, th, pretty, good, right, hello, hello, chicago, nice, see, thank, nice, thank, look, wonderful, crowd, n..."
louis,"[intro, fade, music, let, roll, hold, lights, lights, thank, thank, much, appreciate, necessarily, agree, appreciate, much, well, nice, place, eas..."
mike,"[wow, hey, thank, thanks, thank, guys, hey, seattle, nice, see, look, look, us, crazy, insane, five, years, ago, pretty, much, everyone, know, sta..."
ricky,"[hello, hello, great, thank, wow, calm, shut, fuck, thank]"


## Organizing The Data

### Assignment:
1. Organized data in two standard text formats:
   a) Corpus - corpus is a collection of texts, and they are all put together neatly in a pandas dataframe here.
   b) Document-Term Matrix - word counts in matrix format

### Corpus: Example

A corpus is a collection of texts, and they are all put together neatly in a pandas dataframe here.

In [22]:
# Let's take a look at our dataframe
data_df

,transcript
ali,"Ladies and gentlemen, please welcome to the stage: Ali Wong! Hi. Hello! Welcome! Thank you! Thank you for coming. Hello! Hello. We are gonna have ..."
anthony,"Thank you. Thank you. Thank you, San Francisco . Thank you so much. So good to be here. People were surprised when I told ’em I was gonna tape my..."
bill,"[cheers and applause] All right, thank you! Thank you very much! Thank you. Thank you. Thank you. How are you? What’s going on? Thank you. It’s a ..."
dave,This is Dave. He tells dirty jokes for a living. That stare is where most of his hard work happens. It signifies a profound train of thought...
jim,"[Car horn honks] [Audience cheering] [Announcer] Ladies and gentlemen, please welcome to the stage Mr. Jim Jefferies! [Upbeat music playing] Hello..."
joe,[rock music playing]
john,"All right, Petunia. Wish me luck out there. You will die on August 7th, 2037. That’s pretty good. All right. Hello. Hello, Chicago. Nice to see yo..."
louis,Intro \nFade the music out. Let’s roll. Hold there. Lights. Do the lights. Thank you. Thank you very much. I appreciate that. I don’t necessarily...
mike,"Wow. Hey, thank you. Thanks. Thank you, guys. Hey, Seattle. Nice to see you. Look at this. Look at us. We’re here. This is crazy. It’s insane. So ..."
ricky,Hello. Hello! How you doing? Great. Thank you. Wow. Calm down. Shut the fuck up. Thank you.


In [18]:
# Let's add the comedians' full names as well
full_names = ['Ali Wong', 'Anthony Jeselnik', 'Bill Burr', 'Dave Chappelle',
              'Jim Jefferies', 'Joe Rogan', 'John Mulaney', 'Louis C.K.', 'Mike Birbiglia', 'Ricky Gervais']

data_df['full_name'] = full_names
data_df

,transcript,full_name
ali,"Ladies and gentlemen, please welcome to the stage: Ali Wong! Hi. Hello! Welcome! Thank you! Thank you for coming. Hello! Hello. We are gonna have ...",Ali Wong
anthony,"Thank you. Thank you. Thank you, San Francisco . Thank you so much. So good to be here. People were surprised when I told ’em I was gonna tape my...",Anthony Jeselnik
bill,"[cheers and applause] All right, thank you! Thank you very much! Thank you. Thank you. Thank you. How are you? What’s going on? Thank you. It’s a ...",Bill Burr
dave,This is Dave. He tells dirty jokes for a living. That stare is where most of his hard work happens. It signifies a profound train of thought...,Dave Chappelle
jim,"[Car horn honks] [Audience cheering] [Announcer] Ladies and gentlemen, please welcome to the stage Mr. Jim Jefferies! [Upbeat music playing] Hello...",Jim Jefferies
joe,[rock music playing],Joe Rogan
john,"All right, Petunia. Wish me luck out there. You will die on August 7th, 2037. That’s pretty good. All right. Hello. Hello, Chicago. Nice to see yo...",John Mulaney
louis,Intro \nFade the music out. Let’s roll. Hold there. Lights. Do the lights. Thank you. Thank you very much. I appreciate that. I don’t necessarily...,Louis C.K.
mike,"Wow. Hey, thank you. Thanks. Thank you, guys. Hey, Seattle. Nice to see you. Look at this. Look at us. We’re here. This is crazy. It’s insane. So ...",Mike Birbiglia
ricky,Hello. Hello! How you doing? Great. Thank you. Wow. Calm down. Shut the fuck up. Thank you.,Ricky Gervais


In [24]:
# Let's pickle it for later use
data_df.to_pickle("corpus.pkl")

### Document-Term Matrix: Example

For many of the techniques we'll be using in future assignment, the text must be tokenized, meaning broken down into smaller pieces. The most common tokenization technique is to break down text into words. We can do this using scikit-learn's ` CountVectorizer `, where every row will represent a different document and every column will represent a different word.

In addition, with ` CountVectorizer `, we can remove stop words. Stop words are common words that add no additional meaning to text such as 'a', 'the', etc.

In [19]:
# We are going to create a document-term matrix using CountVectorizer, and exclude common English stop words
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words='english')
data_cv = cv.fit_transform(data_clean.transcript)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = data_clean.index
data_dtm

c:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,aaaaah,abcs,able,absolute,absolutely,absorb,acceptable,accidentally,achieve,act,...,yoga,york,youtube,yummy,ze,zero,zillion,zombie,zombies,zoning
ali,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
anthony,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
bill,1,1,1,1,3,1,1,2,1,4,...,1,1,1,1,1,1,1,1,1,1
dave,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
jim,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
joe,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
john,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
louis,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
mike,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ricky,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
# Let's pickle it for later use
data_dtm.to_pickle("dtm.pkl")

In [21]:
# Let's also pickle the cleaned data (before we put it in document-term matrix format) and the CountVectorizer object
data_clean.to_pickle('data_clean.pkl')
pickle.dump(cv, open("cv.pkl", "wb"))

## Additional Assignments:

1. Can you add an additional regular expression to the clean_text_round2 function to further clean the text?
2. Play around with CountVectorizer's parameters. What is ngram_range? What is min_df and max_df?

Added re.sub(r'[^a-zA-Z]', '', text) to remove everything other than letters

In [22]:
cv = CountVectorizer(stop_words='english', ngram_range=(1,2)) # means unigrams and bigrams
data_cv = cv.fit_transform(data_clean.transcript)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = data_clean.index
data_dtm

c:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,aaaaah,aaaaah anybody,abcs,abcs know,able,able trying,absolute,absolute clusterfuck,absolutely,absolutely kills,...,zero,zero fat,zillion,zillion bucks,zombie,zombie coming,zombies,zombies got,zoning,zoning laws
ali,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
anthony,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
bill,1,1,1,1,1,1,1,1,3,1,...,1,1,1,1,1,1,1,1,1,1
dave,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
jim,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
joe,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
john,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
louis,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
mike,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ricky,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
cv = CountVectorizer(stop_words='english', max_df=4)
data_cv = cv.fit_transform(data_clean.transcript)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = data_clean.index
data_dtm

c:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,aaaaah,abcs,able,absolute,absolutely,absorb,acceptable,accidentally,achieve,act,...,yoga,york,youtube,yummy,ze,zero,zillion,zombie,zombies,zoning
ali,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
anthony,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
bill,1,1,1,1,3,1,1,2,1,4,...,1,1,1,1,1,1,1,1,1,1
dave,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
jim,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
joe,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
john,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
louis,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
mike,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ricky,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
cv = CountVectorizer(stop_words=nltk.corpus.stopwords.words('english'), min_df=0.2)
data_cv = cv.fit_transform(data_clean.transcript)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = data_clean.index
data_dtm

c:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,ago,agree,ah,along,amazing,another,appreciate,around,asleep,ass,...,went,wish,women,work,world,would,wow,yeah,years,york
ali,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
anthony,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
bill,3,1,2,3,1,4,0,25,1,7,...,9,1,5,8,8,20,1,67,18,1
dave,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
jim,0,0,1,1,1,2,2,2,1,0,...,7,0,4,0,0,1,0,1,2,0
joe,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
john,0,0,0,0,0,0,0,0,0,3,...,0,1,0,0,0,0,0,1,0,0
louis,0,1,0,0,0,0,2,1,0,0,...,0,0,0,0,0,0,0,0,0,1
mike,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,3,1,0,1,0
ricky,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


#### Additional Analysis

Count Vectorize transcript

In [31]:
vec = cv.fit(data_clean.transcript)
vec

CountVectorizer(min_df=0.2,
                stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours',
                            'ourselves', 'you', "you're", "you've", "you'll",
                            "you'd", 'your', 'yours', 'yourself', 'yourselves',
                            'he', 'him', 'his', 'himself', 'she', "she's",
                            'her', 'hers', 'herself', 'it', "it's", 'its',
                            'itself', ...])

Creating bag of words from count vector

In [32]:
bow = vec.transform(data_clean.transcript)
bow

<10x264 sparse matrix of type '<class 'numpy.int64'>'
	with 627 stored elements in Compressed Sparse Row format>

Sum of Words created from bag of words

In [33]:
sow = bow.sum(axis=0)
sow

matrix([[  4,   2,   3,   4,   2,   6,   4,  28,   2,  10,   9,   4,  31,
          15,   3,   3,  15,   3,   7,   2,  11,   3,   6,   3,  12,  12,
           7,  21,   4,   3,   6,   7,  28,  13,  18,   2,   3,   2,   6,
           7,   3,   3,   5,  41,   8,   8,   8,   4,  23,  20,  19,  23,
           3,   3,   5,   4,   2,   4,   2,  12,   3,  11,   2,   6,   2,
          10,   2,  39,   7,  72,   3,   2,  72,  17,   5,  15,   3,  62,
          20,  20,  28,   3,  80,  13,  73,   7,  17,  35,  20,   6,   6,
           7,   8,   9,   7,   3,   8,  23,   5,   4,   8,   3,   8,   3,
           5,   2,  13,   2,  14, 108,   2,  11,   2,   3,   3,  34,  24,
         213,  25,   9,   6,  20,  10,  20,  13,  23,   3,   5,   5,   5,
           6,   6,   3,   8,   4,   4,   3,   2,   8,   5,  11,   8,   3,
          12,  28,   6,  12,  13,   2,   2,   2,  21,  24,  49,   9,   6,
           3,  10,  39,   6,   2,   6,   3,   2,   5,   4,   7,  14,   5,
          13,   2,   4,   3, 146,   2,

Arranging words in decreasing order of frequency

In [34]:
word_freq = [(word, sow[0, ind]) for word, ind in vec.vocabulary_.items()]
word_freq = sorted(word_freq, key=lambda x: x[1], reverse=True)
word_freq = pd.DataFrame(word_freq)
word_freq.head(30)

,0,1
0,like,213
1,right,146
2,know,108
3,gonna,80
4,got,73
5,get,72
6,fucking,72
7,yeah,69
8,shit,67
9,go,62


Sentiment Intensity Analysis on Speech

In [35]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer


In [36]:
sia = SentimentIntensityAnalyzer()

for i in data_df.transcript:
    ss = sia.polarity_scores(i)
    print(ss)

{'neg': 0.046, 'neu': 0.685, 'pos': 0.269, 'compound': 0.8655}
{'neg': 0.0, 'neu': 0.718, 'pos': 0.282, 'compound': 0.9438}
{'neg': 0.133, 'neu': 0.733, 'pos': 0.134, 'compound': -0.999}
{'neg': 0.268, 'neu': 0.662, 'pos': 0.07, 'compound': -0.8948}
{'neg': 0.083, 'neu': 0.744, 'pos': 0.173, 'compound': 0.9985}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.081, 'neu': 0.715, 'pos': 0.204, 'compound': 0.9833}
{'neg': 0.068, 'neu': 0.677, 'pos': 0.255, 'compound': 0.9803}
{'neg': 0.087, 'neu': 0.785, 'pos': 0.127, 'compound': 0.9399}
{'neg': 0.117, 'neu': 0.367, 'pos': 0.517, 'compound': 0.8999}
